In [1]:
%cd ../search/search
#! python pacman.py --layout originalClassic

c:\Users\Gabriel de Freitas\OneDrive\Documentos\Tarefas\Universidade\MC886\projects_MachineLearning\project3_ra214129_ra216179\search\search


In [11]:
from pacman import runGames, readCommand
from game import Agent, Directions

In [24]:
class Qlearn(Agent):
    
    def __init__(self, Qtable = []):
        self.Qtable = Qtable
        print("Batata")
    
    def getAction(self, state):
        legal = state.getLegalPacmanActions()
        current = state.getPacmanState().configuration.direction
        if current == Directions.STOP: current = Directions.NORTH
        left = Directions.LEFT[current]
        if left in legal: return left
        if current in legal: return current
        if Directions.RIGHT[current] in legal: return Directions.RIGHT[current]
        if Directions.LEFT[left] in legal: return Directions.LEFT[left]
        return Directions.STOP

In [26]:
agente = Qlearn()
args = readCommand(['-q', '--pacman', 'LeftTurnAgent'])
args["pacman"] = agente

Batata
['-q', '--pacman', 'LeftTurnAgent']


In [27]:
a = runGames(**args)
print(a)

Pacman died! Score: 595
Average Score: 595.0
Scores:        595.0
Win Rate:      0/1 (0.00)
Record:        Loss


In [9]:
print(a[0].state.isWin())

False
